# Data preparation

In [65]:
from datasets import load_from_disk

def new_column(example):
    example["ner_tags"] = example["labels"]
    return example

data = load_from_disk("test.hf")
id_column = range(data.num_rows)
data = data.add_column("id", id_column)
data = data.map(new_column)
data = data.train_test_split(test_size=0.2)
data

# from datasets import load_dataset

# data = load_dataset("wnut_17")
data["train"][0]

# label_list = data["train"].features[f"ner_tags"].feature.names
# label_list

Map:   0%|          | 0/423 [00:00<?, ? examples/s]

{'tokens': ['Tom', 'is', 'as', 'fast', 'as', 'a', 'leopard', '.'],
 'labels': [4, 0, 0, 0, 0, 0, 0, 0],
 'id': 313,
 'ner_tags': [4, 0, 0, 0, 0, 0, 0, 0]}

In [66]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

example = data["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]', 'tom', 'is', 'as', 'fast', 'as', 'a', 'leopard', '.', '[SEP]']

In [67]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_data = data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/338 [00:00<?, ? examples/s]

Map:   0%|          | 0/85 [00:00<?, ? examples/s]

In [68]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluation method

In [69]:
import evaluate
import numpy as np

seqeval = evaluate.load("seqeval")

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Training

In [70]:
# wnut info
# id2label = {
#     0: "O",
#     1: "B-corporation",
#     2: "I-corporation",
#     3: "B-creative-work",
#     4: "I-creative-work",
#     5: "B-group",
#     6: "I-group",
#     7: "B-location",
#     8: "I-location",
#     9: "B-person",
#     10: "I-person",
#     11: "B-product",
#     12: "I-product",
# }

# label2id = {
#     "O": 0,
#     "B-corporation": 1,
#     "I-corporation": 2,
#     "B-creative-work": 3,
#     "I-creative-work": 4,
#     "B-group": 5,
#     "I-group": 6,
#     "B-location": 7,
#     "I-location": 8,
#     "B-person": 9,
#     "I-person": 10,
#     "B-product": 11,
#     "I-product": 12,
# }

# wnut info
id2label = {
    0: "O",
    1: "Agent",
    2: "Location",
    3: "Patient",
    4: "Theme",
    5: "Topic",
    6: "Destination",
    7: "Result",
}

label2id = {
    "O": 0,
    "Agent": 1,
    "Location": 2,
    "Patient": 3,
    "Theme": 4,
    "Topic": 5,
    "Destination": 6,
    "Result": 7,
}

In [71]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=8, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [78]:
# Training parameters
training_args = TrainingArguments(
    output_dir="thematic_role_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.741413,0.000000,0.000000,0.000000,0.764000
2,No log,0.646954,0.294118,0.053191,0.090090,0.768000
3,No log,0.598358,0.370968,0.244681,0.294872,0.772000
4,No log,0.576823,0.384615,0.265957,0.314465,0.780000
5,No log,0.571104,0.375000,0.255319,0.303797,0.778000


C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F

TrainOutput(global_step=110, training_loss=0.6031374844637785, metrics={'train_runtime': 138.9055, 'train_samples_per_second': 12.167, 'train_steps_per_second': 0.792, 'total_flos': 6185768345760.0, 'train_loss': 0.6031374844637785, 'epoch': 5.0})

# Inference

In [83]:
from transformers import pipeline

text = "A dog chases a cat to the bus."

classifier = pipeline("ner", model="thematic_role_model/checkpoint-110")
classifier(text)

[{'entity': 'Agent',
  'score': 0.41819435,
  'index': 1,
  'word': 'a',
  'start': 0,
  'end': 1},
 {'entity': 'Theme',
  'score': 0.36844808,
  'index': 2,
  'word': 'dog',
  'start': 2,
  'end': 5},
 {'entity': 'Theme',
  'score': 0.53722537,
  'index': 4,
  'word': 'a',
  'start': 13,
  'end': 14},
 {'entity': 'Theme',
  'score': 0.55206406,
  'index': 5,
  'word': 'cat',
  'start': 15,
  'end': 18},
 {'entity': 'Theme',
  'score': 0.24746658,
  'index': 7,
  'word': 'the',
  'start': 22,
  'end': 25},
 {'entity': 'Theme',
  'score': 0.34320387,
  'index': 8,
  'word': 'bus',
  'start': 26,
  'end': 29}]

In [85]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("thematic_role_model/checkpoint-110")
inputs = tokenizer(text, return_tensors="pt")

from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("thematic_role_model/checkpoint-110")
with torch.no_grad():
    logits = model(**inputs).logits

predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

print(text)
print(inputs)
print(predicted_token_class)

A dog chases a cat to the bus.
{'input_ids': tensor([[  101,  1037,  3899, 29515,  1037,  4937,  2000,  1996,  3902,  1012,
           102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
['O', 'Agent', 'Theme', 'O', 'Theme', 'Theme', 'O', 'Theme', 'Theme', 'O', 'O']
